<a href="https://colab.research.google.com/github/bohyeon9888/TypingMetric/blob/main/letter%2Btimetaken%2BtotalMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Import

In [ ]:
%config Completer.use_jedi = False
import json
import csv
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dataframe의 전체 열을 보기 위한 작업

In [ ]:
pd.set_option('display.max_columns', None)

## Typing Letter 및 Time Taken 추출

Typing letter을 기록한 데이터프레임 letterdf와 timetaken을 기록한 데이터프레임 timetakendf

In [ ]:
#typing letter 개수를 세기위한 dataframe
letterdf = pd.DataFrame()
letterdf['tester'] = np.nan
letterdf['packageName'] = np.nan
letterdf['counting'] = np.nan

In [ ]:
#timetaken을 기록하기 위한 dataframe
timetakendf = pd.DataFrame()
timetakendf['tester'] = np.nan
timetakendf['packageName'] = np.nan
timetakendf['timeTaken'] = np.nan

### Main 반복문 - 전체 파일을 반복하여 위의 작업 수행해 각 tester 별, package 별 typing letter 및 time taken 추출

In [ ]:
#모든 tester 파일로 동일작업 수행
for i in range(1, 21):
    file = f"C:/Users/dongh/abc/thesis/tester{i}_wsklde.csv"
    df = pd.read_csv(file)
    df['packageName'] = df['packageName'].str.strip()
    df['className'] = df['className'].str.strip()

    #모든 package를 상대로 찾아야 하므로 모든 package를 list로 만듦
    packageArr = df['packageName'].unique()
    tester = f"tester{i}"

    #tester 별 typing letter, time taken 리스트가 추출된 dataframe 가져옴
    cal_package, countingdf = package_get(packageArr, df, tester)

    #전체 timetakendf와 letterdf에 추가함.
    timetakendf = timetakendf.append(cal_package)
    timetakendf['tester'] = timetakendf['tester'].fillna(tester)

    letterdf = letterdf.append(countingdf)
    letterdf['tester'] = letterdf['tester'].fillna(tester)

    del file, df, packageArr, tester, cal_package, countingdf

각 패키지의 time taken의 아웃라이어를 제거하기 위한 함수

In [ ]:
#IQR을 사용한 각 패키지의 timetaken의 아웃라이어 제거
def get_outlier(nonan): #nonan은 Pandas Series 타입

    Q1 = nonan.quantile(.25)
    Q3 = nonan.quantile(.75)

    IQR = Q3 - Q1
    IQR_weight = IQR * 1.5

    lowest = Q1 - IQR_weight #아웃라이어 제거를 위한 하방값
    highest = Q3 + IQR_weight #아웃라이어 제거를 위한 상방값
    del nonan, IQR_weight, IQR

    return lowest, highest

### 각 패키지마다 각 view의 typing letter을 세기위한 작업

In [ ]:
#typing letter 세기
def letter_count(ind_list, df):
    #각 view마다 typing letter의 개수를 세서 담기 위한 리스트 'counting'
    counting = []

    #각 view마다 time taken을 추출하기 위한 시리즈'packagser'
    packagser = pd.Series().astype(int)

    for j in range(len(ind_list)): #package_get에서 가져온 index list인 'ind_list'
        start_index = ind_list[j] #하나의 view의 시작점 'start_index'
        if j+1 >= len(ind_list):
            del start_index
            break;
        end_index = ind_list[j+1] #view의 종결점 'end_index'
        #시작점과 종결점 사이 KEY LOG만 수집해서 new_df 만듦
        new_df = df.loc[(df.index >= start_index) & (df.index <= end_index) & (df['datumType'] == 'KEY_LOG')]
        if new_df.empty:
            del start_index, end_index, new_df
            continue;

        new_df = new_df.sort_index()
        new_df['timeTaken'] = pd.to_numeric(new_df['timeTaken'], errors='coerce').fillna(0).astype(float)
        bufser = new_df.loc[:, 'timeTaken'] #new_df의 timetaken들을 임시 series에 저장
        if bufser.empty:
            del start_index, end_index, new_df, bufser
            continue;

        #typing letter에서 timetaken이 0 이하면 오류이므로 제거하고 글자수 세기
        lettseries = bufser[bufser>0]
        letter = len(lettseries) #typing letter 개수를 letter로 설정
        #한 view의 timetaken 기록
        bufser = bufser.reset_index(drop=True)
        bufser = bufser.drop(0, axis=0) #timetaken 임시 시리즈에서 맨 처음 값 제거

        packagser = packagser.append(bufser) #최종 timetaken 추출본인 packagser에 bufser 붙이기

        if letter is None: #letter에 아무것도 없으면 counting에 0 추가(0 대신 None이 기록될 때도 있음)
            counting.append(0)
        else:
            counting.append(letter)

        del start_index, end_index, new_df, lettseries, bufser

    #timetaken outlier를 제거
    lowest, highest = get_outlier(packagser)
    if lowest <= 0: #하방값이 0이하면 packagser에서 0 초과 상방값 이하 값들만 저장
        packagser = packagser[(packagser > 0) & (packagser<=highest)]
    else:
        packagser = packagser[(packagser >=lowest) & (packagser<=highest)]

    #시리즈 packagser와 리스트 counting 값 내보내기
    return packagser, counting

### 각 package list를 가져와 각 package의 view들을 찾기 위한 loopdf 만드는 작업

In [ ]:
def package_get(packageArr, df, tester):
    #각 tester의 package 별 time taken 리스트를 저장하기 위한 Dataframe
    cal_package = pd.DataFrame()
    cal_package['tester'] = np.nan
    cal_package['packageName'] = np.nan
    cal_package['timeTaken'] = np.nan

    #각 tester의 package 별 typing letter 리스트를 저장하기 위한 Dataframe
    countingdf = pd.DataFrame()
    countingdf['tester'] = np.nan
    countingdf['packageName'] = np.nan
    countingdf['counting'] = np.nan

    for package in packageArr: #packageArr는 각 tester 별 모든 package의 list, main 반복문에서 나옴
        #bufferdf는 해당 package에 해당하는 모든 행들과 Device Event를 가져옴
        bufferdf = df[(df['packageName']==package) | (df['datumType'] == 'DEVICE_EVENT')]
        if bufferdf['timeTaken'].isnull().all():
            del bufferdf
            continue;

        #각 view의 시작점 및 종결점 잡기 위해 External Sensor 중 'typeWindowStateChanged', 'typeViewFocused'인 행들을 추출한 classdf
        classdf = bufferdf.loc[(bufferdf['deviceType'].isin(['typeWindowStateChanged', 'typeViewFocused'])),['timestamp', 'packageName', 'deviceType', 'className', 'type']]
        #각 view의 시작점 및 종결점 잡기 위해 Device Event 중 'SCREEN_OFF', 'SCREEN_ON', 'USER_PRESENT'인 행들을 추출한 eventdf
        eventdf = bufferdf.loc[(bufferdf['type'].isin(['SCREEN_OFF', 'SCREEN_ON', 'USER_PRESENT'])),['timestamp', 'packageName', 'deviceType', 'className', 'type']]
        #classdf와 eventdf를 합친 loopdf
        loopdf = classdf.append(eventdf)
        if loopdf.empty:
            del bufferdf, eventdf, classdf, loopdf, packageser
            continue;

        loopdf = loopdf.sort_index()
        #loopdf의 index를 가져온 ind_list
        ind_list = loopdf.index

        #time taken을 추출한 시리즈 packageser와 typing letter을 추출한 counting 리스트를 가져오기 위해 letter_count 함수에 ind_list와 bufferdf
        packageser, counting = letter_count(ind_list, bufferdf)

        #timetaken 및 counting 리스트 형태로 dataframe에 저장
        newRow_time = {'tester':tester, 'packageName': package, 'timeTaken': packageser.tolist()}
        newRow_count = {'tester':tester, 'packageName': package, 'counting': counting}

        cal_package = cal_package.append(newRow_time, ignore_index=True)
        countingdf = countingdf.append(newRow_count, ignore_index=True)

        del bufferdf, ind_list, classdf, eventdf, loopdf, packageser, counting, newRow_time, newRow_count

    #typing list 및 time taken을 추출한 Dataframe 반환
    return cal_package, countingdf

In [ ]:
#같은 tester가 중복의 package를 가지고 있을 때 합치기
totaltimedf = timetakendf.groupby(['packageName', 'tester']).agg({'timeTaken':'sum'}).reset_index()

In [ ]:
#리스트가 없는 package, tester 행 없앰
totaltimedf = totaltimedf[totaltimedf['timeTaken'].apply(lambda x: len(x)>0)]

In [ ]:
#letterdf도 동일 작업
totaltypingdf = letterdf.groupby(['packageName', 'tester']).agg({'counting':'sum'}).reset_index()
totaltypingdf = totaltypingdf[totaltypingdf['counting'].apply(lambda x: len(x)>0)]

In [ ]:
#json형태로 저장, 파일 경로: https://drive.google.com/drive/folders/1CUxI_x-wxgF_5LcKMz4MjBkzPsj_168u?usp=sharing
totaltimedf = totaltimedf.reset_index()
totaltimedf.to_json("C:/Users/dongh/abc/thesis/timetaken.json", orient='records', indent=4)
totaltypingdf = letterdf.reset_index()
totaltypingdf.to_json("C:/Users/dongh/abc/thesis/letterdf.json", orient='records', indent=4)

## 각 tester 별 typing metric 구하기
### 각 tester 별로 package 별, class 별 아웃라이어 제거하기 위해 기존에 구해놓은 quantile csv 파일 가져오기

아웃라이어를 제거하기 위한 함수를 쓸 수도 있지만 runtime이 너무 오래걸릴 것 같아 기존에 추출한 파일을 불러오는 것으로 대체

In [ ]:
#전체 인원에 대한 outlier 제거 quantile, 전체 인원에 대한 정상치 상방값 및 하방값
#파일 링크: https://docs.google.com/spreadsheets/d/1yU0bSMlZwO8_0J1lKQOke5ZTg4YXgz-jjId2kU12G7I/edit#gid=1692434663
quantile = "C:/Users/dongh/abc/thesis/total_quantile.csv"

In [ ]:
qdf = pd.read_csv(quantile)

In [ ]:
qdf

,mean,min,max,median,Q1,Q2,Q3,Q4,IQR,lowest,highest,packageName,Pmean,Pmin,Pmax,Pmedian,P_Q1,P_Q2,P_Q3,P_Q4,P_IQR,P_lowest,P_highest,className,Cmean,Cmin,Cmax,Cmedian,C_Q1,C_Q2,C_Q3,C_Q4,C_IQR,C_lowest,C_highest
0,292.9,1.0,953531.0,149.0,91.0,149.0,219.0,953531.0,128.0,-101.0,411.0,Com.sktelecom.minit,177.50,42.0,289.0,196.5,105.75,196.5,251.25,289.0,145.50,-112.50,469.50,android.webkit.WebView,177.50,42.0,289.0,196.5,105.75,196.5,251.25,289.0,145.5,-112.50,469.50
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ai.mindlogic.dueta,233.67,144.0,600.0,187.0,171.50,187.0,241.50,600.0,70.00,66.50,346.50,ai.mindlogic.dueta.ui.chat.view.ChatRoomActivity,236.32,144.0,600.0,187.0,173.50,187.0,253.00,600.0,79.5,54.25,372.25
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ai.mindlogic.dueta,233.67,144.0,600.0,187.0,171.50,187.0,241.50,600.0,70.00,66.50,346.50,android.widget.EditText,227.38,162.0,468.0,189.5,167.25,189.5,238.25,468.0,71.0,60.75,344.75
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cc.dreamspark.intervaltimer,261.79,2.0,3578.0,142.0,65.25,142.0,245.50,3578.0,180.25,-205.12,515.88,android.widget.EditText,269.50,2.0,3578.0,142.0,62.75,142.0,241.25,3578.0,178.5,-205.00,509.00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cc.dreamspark.intervaltimer,261.79,2.0,3578.0,142.0,65.25,142.0,245.50,3578.0,180.25,-205.12,515.88,android.widget.FrameLayout,192.38,3.0,540.0,129.0,99.00,129.0,265.50,540.0,166.5,-150.75,515.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,viva.republica.toss,367.69,1.0,2786.0,229.0,141.00,229.0,397.00,2786.0,256.00,-243.00,781.00,android.widget.EditText,362.14,1.0,2501.0,219.0,136.50,219.0,404.00,2501.0,267.5,-264.75,805.25
916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,viva.republica.toss,367.69,1.0,2786.0,229.0,141.00,229.0,397.00,2786.0,256.00,-243.00,781.00,android.widget.FrameLayout,443.95,2.0,2786.0,245.0,145.00,245.0,555.00,2786.0,410.0,-470.00,1170.00
917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,viva.republica.toss,367.69,1.0,2786.0,229.0,141.00,229.0,397.00,2786.0,256.00,-243.00,781.00,viva.republica.toss.send.v4.receiver.account.I...,362.38,51.0,1390.0,298.0,215.00,298.0,407.00,1390.0,192.0,-73.00,695.00
918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witplus.clio.starter,592.17,150.0,1525.0,459.0,332.00,459.0,675.00,1525.0,343.00,-182.50,1189.50,android.webkit.WebView,496.79,150.0,1525.0,391.5,293.75,391.5,472.75,1525.0,179.0,25.25,741.25


각 package의 class 별 typing metric을 저장하기 위한 Dataframe

In [ ]:
sumdf = pd.DataFrame()
sumdf['packageName'] = np.nan
sumdf['className'] = np.nan
sumdf['CavgCPS'] = np.nan
sumdf['CminCPS'] = np.nan
sumdf['CmidCPS'] = np.nan
sumdf['CmaxCPS'] = np.nan
sumdf['CavgCPM'] = np.nan
sumdf['CminCPM'] = np.nan
sumdf['CmidCPM'] = np.nan
sumdf['CmaxCPM'] = np.nan
sumdf['CavgWPS'] = np.nan
sumdf['CminWPS'] = np.nan
sumdf['CmidWPS'] = np.nan
sumdf['CmaxWPS'] = np.nan
sumdf['CavgWPM'] = np.nan
sumdf['CminWPM'] = np.nan
sumdf['CmidWPM'] = np.nan
sumdf['CmaxWPM'] = np.nan
sumdf['CavgKSPS'] = np.nan
sumdf['CminKSPS'] = np.nan
sumdf['CmidKSPS'] = np.nan
sumdf['CmaxKSPS'] = np.nan
sumdf['CavgKSPC'] = np.nan
sumdf['CminKSPC'] = np.nan
sumdf['CmidKSPC'] = np.nan
sumdf['CmaxKSPC'] = np.nan
sumdf['CavgIntra'] = np.nan
sumdf['CminIntra'] = np.nan
sumdf['CmidIntra'] = np.nan
sumdf['CmaxIntra'] = np.nan
sumdf['CavgInter'] = np.nan
sumdf['CminInter'] = np.nan
sumdf['CmidInter'] = np.nan
sumdf['CmaxInter'] = np.nan
sumdf

,packageName,className,CavgCPS,CminCPS,CmidCPS,CmaxCPS,CavgCPM,CminCPM,CmidCPM,CmaxCPM,CavgWPS,CminWPS,CmidWPS,CmaxWPS,CavgWPM,CminWPM,CmidWPM,CmaxWPM,CavgKSPS,CminKSPS,CmidKSPS,CmaxKSPS,CavgKSPC,CminKSPC,CmidKSPC,CmaxKSPC,CavgIntra,CminIntra,CmidIntra,CmaxIntra,CavgInter,CminInter,CmidInter,CmaxInter


각 tester 별 pacakge의 typing metric을 구하기 위한 Dataframe

In [ ]:
packagedf = pd.DataFrame()
packagedf['tester'] = np.nan
packagedf['packageName'] = np.nan
packagedf['PavgCPS'] = np.nan
packagedf['PminCPS'] = np.nan
packagedf['PmidCPS'] = np.nan
packagedf['PmaxCPS'] = np.nan
packagedf['PavgCPM'] = np.nan
packagedf['PminCPM'] = np.nan
packagedf['PmidCPM'] = np.nan
packagedf['PmaxCPM'] = np.nan
packagedf['PavgWPS'] = np.nan
packagedf['PminWPS'] = np.nan
packagedf['PmidWPS'] = np.nan
packagedf['PmaxWPS'] = np.nan
packagedf['PavgWPM'] = np.nan
packagedf['PminWPM'] = np.nan
packagedf['PmidWPM'] = np.nan
packagedf['PmaxWPM'] = np.nan
packagedf['PavgKSPS'] = np.nan
packagedf['PminKSPS'] = np.nan
packagedf['PmidKSPS'] = np.nan
packagedf['PmaxKSPS'] = np.nan
packagedf['PavgKSPC'] = np.nan
packagedf['PminKSPC'] = np.nan
packagedf['PmidKSPC'] = np.nan
packagedf['PmaxKSPC'] = np.nan
packagedf['PavgIntra'] = np.nan
packagedf['PminIntra'] = np.nan
packagedf['PmidIntra'] = np.nan
packagedf['PmaxIntra'] = np.nan
packagedf['PavgInter'] = np.nan
packagedf['PminInter'] = np.nan
packagedf['PmidInter'] = np.nan
packagedf['PmaxInter'] = np.nan
packagedf

,tester,packageName,PavgCPS,PminCPS,PmidCPS,PmaxCPS,PavgCPM,PminCPM,PmidCPM,PmaxCPM,PavgWPS,PminWPS,PmidWPS,PmaxWPS,PavgWPM,PminWPM,PmidWPM,PmaxWPM,PavgKSPS,PminKSPS,PmidKSPS,PmaxKSPS,PavgKSPC,PminKSPC,PmidKSPC,PmaxKSPC,PavgIntra,PminIntra,PmidIntra,PmaxIntra,PavgInter,PminInter,PmidInter,PmaxInter


각 테스터 별 모든 time taken의 typing metric을 구하기 위한 Dataframe

In [ ]:
testerdf = pd.DataFrame()
testerdf['tester'] = np.nan
testerdf['avgCPS'] = np.nan
testerdf['minCPS'] = np.nan
testerdf['midCPS'] = np.nan
testerdf['maxCPS'] = np.nan
testerdf['avgCPM'] = np.nan
testerdf['minCPM'] = np.nan
testerdf['midCPM'] = np.nan
testerdf['maxCPM'] = np.nan
testerdf['avgWPS'] = np.nan
testerdf['minWPS'] = np.nan
testerdf['midWPS'] = np.nan
testerdf['maxWPS'] = np.nan
testerdf['avgWPM'] = np.nan
testerdf['minWPM'] = np.nan
testerdf['midWPM'] = np.nan
testerdf['maxWPM'] = np.nan
testerdf['avgKSPS'] = np.nan
testerdf['minKSPS'] = np.nan
testerdf['midKSPS'] = np.nan
testerdf['maxKSPS'] = np.nan
testerdf['avgKSPC'] = np.nan
testerdf['minKSPC'] = np.nan
testerdf['midKSPC'] = np.nan
testerdf['maxKSPC'] = np.nan
testerdf['avgIntra'] = np.nan
testerdf['minIntra'] = np.nan
testerdf['midIntra'] = np.nan
testerdf['maxIntra'] = np.nan
testerdf['avgInter'] = np.nan
testerdf['minInter'] = np.nan
testerdf['midInter'] = np.nan
testerdf['maxInter'] = np.nan
testerdf

,tester,avgCPS,minCPS,midCPS,maxCPS,avgCPM,minCPM,midCPM,maxCPM,avgWPS,minWPS,midWPS,maxWPS,avgWPM,minWPM,midWPM,maxWPM,avgKSPS,minKSPS,midKSPS,maxKSPS,avgKSPC,minKSPC,midKSPC,maxKSPC,avgIntra,minIntra,midIntra,maxIntra,avgInter,minInter,midInter,maxInter


각 tester의 전체 typing metric을 구하기 위한 리스트 준비

In [ ]:
tavgCPS = []
tminCPS = []
tmidCPS = []
tmaxCPS = []
tavgCPM = []
tminCPM = []
tmidCPM = []
tmaxCPM = []
tavgWPS = []
tminWPS = []
tmidWPS = []
tmaxWPS = []
tavgWPM = []
tminWPM = []
tmidWPM = []
tmaxWPM = []
tavgKSPS = []
tminKSPS = []
tmidKSPS = []
tmaxKSPS = []
tavgKSPC = []
tminKSPC = []
tmidKSPC = []
tmaxKSPC = []

In [ ]:
avgIntra = []
avgInter = []

각 tester의 EXTERNAL SENSOR, KEY LOG, DEVICE EVENT를 합친 파일 <br>
원본 json파일을 직접 불러와서 가공하고 바로 합쳐서 쓸 수 있지만 runtime이 오래 걸리고 약 5기가 이상의 파일을 처리하면 주피터 노트북이 멈추는 현상이 발생해 미리 가공해 놓은 파일 사용<br>
파일링크는 위에 있음<br>

In [ ]:
file = "C:/Users/dongh/abc/thesis/tester6_wsklde.csv"

In [ ]:
tester = 'tester6'

In [ ]:
df=pd.read_csv(file)

In [ ]:
df['packageName'] = df['packageName'].str.strip()
df['className'] = df['className'].str.strip()

tester의 모든 package를 체크하기 위한 package list인 packageArr를 만듦

In [ ]:
packageArr = df['packageName'].unique()

### 각 class 별 typing metric을 구하기 위한 get_caver 함수

In [ ]:
#class 단위로 typing metric 구하기
def get_caver(df, loopdf, classArr):
    sumdf = pd.DataFrame() #class 별 typing metric을 구하기 위한 dataframe 및 list
    sumdf['packageName'] = np.nan
    sumdf['className'] = np.nan
    sumdf['CavgCPS'] = np.nan
    sumdf['CminCPS'] = np.nan
    sumdf['CmidCPS'] = np.nan
    sumdf['CmaxCPS'] = np.nan
    sumdf['CavgCPM'] = np.nan
    sumdf['CminCPM'] = np.nan
    sumdf['CmidCPM'] = np.nan
    sumdf['CmaxCPM'] = np.nan
    sumdf['CavgWPS'] = np.nan
    sumdf['CminWPS'] = np.nan
    sumdf['CmidWPS'] = np.nan
    sumdf['CmaxWPS'] = np.nan
    sumdf['CavgWPM'] = np.nan
    sumdf['CminWPM'] = np.nan
    sumdf['CmidWPM'] = np.nan
    sumdf['CmaxWPM'] = np.nan
    sumdf['CavgKSPS'] = np.nan
    sumdf['CminKSPS'] = np.nan
    sumdf['CmidKSPS'] = np.nan
    sumdf['CmaxKSPS'] = np.nan
    sumdf['CavgKSPC'] = np.nan
    sumdf['CminKSPC'] = np.nan
    sumdf['CmidKSPC'] = np.nan
    sumdf['CmaxKSPC'] = np.nan
    sumdf['CavgIntra'] = np.nan
    sumdf['CminIntra'] = np.nan
    sumdf['CmidIntra'] = np.nan
    sumdf['CmaxIntra'] = np.nan
    sumdf['CavgInter'] = np.nan
    sumdf['CminInter'] = np.nan
    sumdf['CmidInter'] = np.nan
    sumdf['CmaxInter'] = np.nan

    pavgCPS = []
    pminCPS = []
    pmidCPS = []
    pmaxCPS = []
    pavgCPM = []
    pminCPM = []
    pmidCPM = []
    pmaxCPM = []
    pavgWPS = []
    pminWPS = []
    pmidWPS = []
    pmaxWPS = []
    pavgWPM = []
    pminWPM = []
    pmidWPM = []
    pmaxWPM = []
    pavgKSPS = []
    pminKSPS = []
    pmidKSPS = []
    pmaxKSPS = []
    pavgKSPC = []
    pminKSPC = []
    pmidKSPC = []
    pmaxKSPC = []
    package = loopdf.iloc[0, 1]

    for i in range(len(classArr)): #한 package의 모든 class를 가져온 classArr에서 class 이름 가져옴
        avgCPS = []
        avgCPM = []
        avgWPS = []
        avgWPM = []
        avgKSPS = []
        avgKSPC = []
        avg_Intra = []
        avg_Inter = []

        class_name = classArr[i]
        #loopdf에서 해당 class에 해당하는 행들을 가져와서 행들의 index 번호로 sub_list 만듦
        sub_list = loopdf[(loopdf['className']==class_name)].index.tolist()

        #해당 패키지 및 클래스에 해당하는 outlier 제거용 상방값(high) 및 하방값(low) 가져옴
        #전체 quantile file에서 해당 class name을 가진 quantile에서 가져옴
        high = qdf.loc[(qdf['className'] == class_name) & (qdf['packageName'] == package)]['C_highest'].reset_index(drop=True)
        if high.empty:
            del class_name, sub_list, high, avgCPS, avgCPM, avgWPS, avgWPM, avgKSPS, avgKSPC, avg_Intra, avg_Inter
            continue;

        low = qdf.loc[qdf['className'] == class_name]['C_lowest'].reset_index(drop=True)
        highest = high[0]
        originlowest = low[0]
        #하방값이 0 이하면 하방값 1로 설정
        if originlowest <= 0:
            lowest = 1
        else:
            lowest = originlowest

        #sub_list 안에서 인덱스 번호로 view의 시작점 및 종결접 잡기
        for j in range(len(sub_list)):
            if sub_list[j] in loopdf.index:
                #시작점
                start_index = sub_list[j]
                #종결점 = 시작점 바로 다음 인덱스 번호
                end_index = loopdf.index.get_loc(start_index) + 1

                if end_index < len(loopdf):
                    end_index = loopdf.index[end_index]
                    #시작점과 종결점 사이 Key Log 추출한 new_df
                    new_df = df.loc[(df.index >= start_index) & (df.index <= end_index) & (df['datumType'] == 'KEY_LOG')]
                    if new_df.empty:
                        del start_index, end_index, new_df
                        continue;

                    #intracharacter time을 구하기 위해 start index와 end index 행을 new_df에 추가
                    new_df = new_df.append(df.loc[[start_index, end_index]])
                    new_df = new_df.sort_index()
                    new_df['timeTaken'] = pd.to_numeric(new_df['timeTaken'], errors='coerce').fillna(0).astype(float)
                    #intercharacter time을 구하기 위해 time taken을 inter_arr에 시리즈로 저장
                    inter_arr = new_df.loc[:, 'timeTaken']
                    inter_arr = inter_arr.dropna()
                    if inter_arr.empty:
                        del start_index, end_index, new_df, inter_arr
                        continue;

                    inter_arr = inter_arr.reset_index(drop=True)

                    #inter_arr에서 맨 처음 값은 전 view부터 가져온 대기값이므로 제거
                    inter_arr = inter_arr.drop(0, axis=0)

                    #inter_arr에서 class의 정상치 하방값 미만 값들 제거
                    inter_arr = inter_arr[inter_arr>=lowest]

                    #inter_arr에서 class의 정상치 상방값 초과 값들 제거
                    inter_arr = inter_arr.drop(inter_arr[inter_arr >= highest].index)

                    #intracharacter time에는 typing이 끝나고 터치를 하며 넘어가는 시간차가 계산에 필요하므로
                    #종결점에서 마지막 typing log timestamp의 차이를 구함
                    last_time = new_df.iloc[-1, 2] - new_df.iloc[-2, 2]
                    intra_arr = inter_arr.append(pd.Series(last_time))

                    #새로운 값이 추가됐으므로 다시 아웃라이어 제거
                    intra_arr = intra_arr[(intra_arr>=lowest) & (intra_arr <= highest)]

                    #backspace 키 개수
                    back_count = len(new_df[new_df['currentKeyType'].notna() & new_df['currentKeyType'].str.contains('BACKSPACE')])
                    countIS = len(new_df) - 2 #한 view에서의 typing 개수
                    word_T = countIS - back_count #한 view의 typing 개수에서 backspace를 뺀 것
                    first_s=new_df.iloc[1, 2] #typing 을 시작한 지점의 timestamp
                    last_s=new_df.iloc[-2, 2] #typing 을 종료한 지점의 timestamp
                    time_dif = round(((last_s - first_s)/1000), 1) #typing을 한 총 시간 (minute 단위)

                    #typing metric 구하기
                    if word_T != 0:
                        CPS = (word_T-1) / time_dif
                        CPM = CPS * 60
                        WPS = CPS / 5
                        WPM = WPS * 60
                        KSPS = countIS / time_dif
                        KSPC = countIS / word_T
                    else:
                        CPS = (word_T-1) / time_dif
                        CPM = CPS * 60
                        WPS = CPS / 5
                        WPM = WPS * 60
                        KSPS = countIS / time_dif
                        KSPC = 0

                    Inter = (inter_arr.sum()) / countIS - 1
                    Intra = (intra_arr.sum()) / countIS

                    #리스트에 각 typing metric 추가
                    avg_Intra.append(Intra)
                    avg_Inter.append(Inter)
                    avgCPS.append(CPS)
                    avgCPM.append(CPM)
                    avgWPS.append(WPS)
                    avgWPM.append(WPM)
                    avgKSPS.append(KSPS)
                    avgKSPC.append(KSPC)

                    del start_index, end_index, inter_arr, intra_arr, new_df

        #typing metric의 통계치를 구하기 위한 전처리
        avg_Intra = [x for x in avg_Intra if x is not None]
        avg_Inter = [x for x in avg_Inter if x is not None]
        avg_Intra = [x for x in avg_Intra if x > 0]
        avg_Inter = [x for x in avg_Inter if x > 0]
        avgCPS = [x for x in avgCPS if x is not None]
        avgCPM = [x for x in avgCPM if x is not None]
        avgWPS = [x for x in avgWPS if x is not None]
        avgWPM = [x for x in avgWPM if x is not None]
        avgKSPS = [x for x in avgKSPS if x is not None]
        avgKSPC = [x for x in avgKSPC if x is not None]
        avgCPS = [x for x in avgCPS if x > 0]
        avgCPM = [x for x in avgCPM if x > 0]
        avgWPS = [x for x in avgWPS if x > 0]
        avgWPM = [x for x in avgWPM if x > 0]
        avgKSPS = [x for x in avgKSPS if x > 0]
        avgKSPC = [x for x in avgKSPC if x > 0]


        if len(avg_Inter) <= 0:
            del sub_list, highest, originlowest, avgCPS, avgCPM, avgWPS, avgWPM, avgKSPS, avgKSPC, avg_Intra, avg_Inter, high, low, lowest
            continue;

        avgCPS = np.array(avgCPS)
        avgCPM = np.array(avgCPM)
        avgWPS = np.array(avgWPS)
        avgWPM = np.array(avgWPM)
        avgKSPS = np.array(avgKSPS)
        avgKSPC = np.array(avgKSPC)

        avgCPS = avgCPS[~np.isnan(avgCPS)]
        avgCPM = avgCPM[~np.isnan(avgCPM)]
        avgWPS = avgWPS[~np.isnan(avgWPS)]
        avgWPM = avgWPM[~np.isnan(avgWPM)]
        avgKSPS = avgKSPS[~np.isnan(avgKSPS)]
        avgKSPC = avgKSPC[~np.isnan(avgKSPC)]

        avgCPS = np.nan_to_num(avgCPS, nan=0, posinf=0, neginf=0)
        avgCPM = np.nan_to_num(avgCPM, nan=0, posinf=0, neginf=0)
        avgWPS = np.nan_to_num(avgWPS, nan=0, posinf=0, neginf=0)
        avgWPM = np.nan_to_num(avgWPM, nan=0, posinf=0, neginf=0)
        avgKSPS = np.nan_to_num(avgKSPS, nan=0, posinf=0, neginf=0)
        avgKSPC = np.nan_to_num(avgKSPC, nan=0, posinf=0, neginf=0)

        if avgCPS.size <= 0:
            del sub_list, highest, originlowest, avgCPS, avgCPM, avgWPS, avgWPM, avgKSPS, avgKSPC, avg_Intra, avg_Inter, high, low, lowest
            continue;

        #통계치를 구해서 sumdf에 추가
        newRow = {'packageName': package, 'className': class_name, 'CavgCPS': np.mean(avgCPS), 'CminCPS': np.min(avgCPS),
                  'CmidCPS': np.median(avgCPS), 'CmaxCPS': np.max(avgCPS), 'CavgCPM': np.mean(avgCPM), 'CminCPM': np.min(avgCPM),
                  'CmidCPM': np.median(avgCPM), 'CmaxCPM': np.max(avgCPM), 'CavgWPS': np.mean(avgWPS), 'CminWPS': np.min(avgWPS),
                  'CmidWPS': np.median(avgWPS), 'CmaxWPS': np.max(avgWPS), 'CavgWPM': np.mean(avgWPM), 'CminWPM': np.min(avgWPM),
                  'CmidWPM': np.median(avgWPM), 'CmaxWPM': np.max(avgWPM),'CavgKSPS': np.mean(avgKSPS), 'CminKSPS': np.min(avgKSPS),
                  'CmidKSPS': np.median(avgKSPS), 'CmaxKSPS': np.max(avgKSPS), 'CavgKSPC': np.mean(avgKSPC), 'CminKSPC': np.min(avgKSPC),
                  'CmidKSPC': np.median(avgKSPC), 'CmaxKSPC': np.max(avgKSPC), 'CavgIntra': np.mean(avg_Intra), 'CminIntra': np.min(avg_Intra),
                  'CmidIntra': np.median(avg_Intra), 'CmaxIntra': np.max(avg_Intra), 'CavgInter': np.mean(avg_Inter),
                  'CminInter': np.min(avg_Inter), 'CmidInter': np.median(avg_Inter), 'CmaxInter': np.max(avg_Inter)}
        sumdf = sumdf.append(newRow, ignore_index=True)

        avgCPS = avgCPS.tolist()
        avgCPM = avgCPM.tolist()
        avgWPS = avgWPS.tolist()
        avgWPM = avgWPM.tolist()
        avgKSPS = avgKSPS.tolist()
        avgKSPC = avgKSPC.tolist()

        #package 별 typing metric을 구하기 위해 package typing metric list에 class 별 값 추가
        pminCPS.append(np.min(avgCPS))
        pmidCPS.append(np.median(avgCPS))
        pmaxCPS.append(np.max(avgCPS))
        pminCPM.append(np.min(avgCPM))
        pmidCPM.append(np.median(avgCPM))
        pmaxCPM.append(np.max(avgCPM))
        pminWPS.append(np.min(avgWPS))
        pmidWPS.append(np.median(avgWPS))
        pmaxWPS.append(np.max(avgWPS))
        pminWPM.append(np.min(avgWPM))
        pmidWPM.append(np.median(avgWPM))
        pmaxWPM.append(np.max(avgWPM))
        pminKSPS.append(np.min(avgKSPS))
        pmidKSPS.append(np.median(avgKSPS))
        pmaxKSPS.append(np.max(avgKSPS))
        pminKSPC.append(np.min(avgKSPC))
        pmidKSPC.append(np.median(avgKSPC))
        pmaxKSPC.append(np.max(avgKSPC))

        pavgCPS = pavgCPS + avgCPS
        pavgCPM = pavgCPM + avgCPM
        pavgWPS = pavgWPS + avgWPS
        pavgWPM = pavgWPM + avgWPM
        pavgKSPS = pavgKSPS + avgKSPS
        pavgKSPC = pavgKSPC + avgKSPC

        del sub_list, highest, originlowest, avgCPS, avgCPM, avgWPS, avgWPM, avgKSPS, avgKSPC, avg_Intra, avg_Inter, high, low, lowest

    return sumdf, pavgCPS, pavgCPM, pavgWPS, pavgWPM, pavgKSPS, pavgKSPC, pminCPS, pmidCPS, pmaxCPS, pminCPM , pmidCPM, pmaxCPM, pminWPS, pmidWPS, pmaxWPS, pminWPM, pmidWPM , pmaxWPM, pminKSPS, pmidKSPS, pmaxKSPS, pminKSPC, pmidKSPC, pmaxKSPC

### 각 package 별 typing metric 구하기

In [ ]:
def get_paver(ind_list, df, highest, lowest):
    avg_Intra = []
    avg_Inter = []
    tavg_Intra = []
    tavg_Inter = []

    for j in range (len(ind_list)):
        #각 view의 시작점 및 종결점 구하기
        start_index = ind_list[j]
        if j+1 >= len(ind_list):
            del start_index
            break;
        end_index = ind_list[j+1]
        new_df = df.loc[(df.index >= start_index) & (df.index <= end_index) & (df['datumType'] == 'KEY_LOG')]
        if new_df.empty:
            del start_index, end_index, new_df
            continue;
        new_df = new_df.append(df.loc[[start_index, end_index]])
        new_df = new_df.sort_index()
        new_df['timeTaken'] = pd.to_numeric(new_df['timeTaken'], errors='coerce').fillna(0).astype(float)
        inter_arr = new_df.loc[:, 'timeTaken']
        inter_arr = inter_arr.dropna()
        if inter_arr.empty:
            del start_index, end_index, new_df, inter_arr
            continue;

        inter_arr = inter_arr.reset_index(drop=True)
        inter_arr = inter_arr.drop(0, axis=0)
        inter_arr1 = inter_arr[(inter_arr>=lowest) & (inter_arr <= highest)] #전체 인원에 대한 package 별 아웃라이어 제거
        inter_arr2 = inter_arr[(inter_arr>=1) & (inter_arr <= 411)] #전체 인원에 대한 아웃라이어 제거, 하드코딩한 부분
        last_time = new_df.iloc[-1, 2] - new_df.iloc[-2, 2]
        intra_arr = inter_arr.append(pd.Series(last_time))
        intra_arr1 = intra_arr[(intra_arr>=lowest) & (intra_arr <= highest)] #전체 인원에 대한 package 별 아웃라이어 제거
        intra_arr2 = intra_arr[(intra_arr>=1) & (intra_arr <= 411)] #전체 인원에 대한 아웃라이어 제거, 하드코딩한 부분

        countIS = len(new_df) - 2

        #package에 대한 각 view의 typing metric(CPS, CPM, WPS, WPM, KSPS, KSPC 등)은 아웃라이어 제거가 필요없으므로 제거가 필요한
        #intracharacter, intercharacter time 등만 구함
        if countIS <= 0:
            Inter = 0
            Intra = 0
        Interp = (inter_arr1.sum()) / countIS - 1
        Intrap = (intra_arr1.sum()) / countIS
        Intert = (inter_arr2.sum()) / countIS - 1
        Intrat = (intra_arr2.sum()) / countIS

        #각 package 별 intra, inter
        avg_Intra.append(Intrap)
        avg_Inter.append(Interp)
        #전체 인원에 대한 intra, inter
        tavg_Intra.append(Intrat)
        tavg_Inter.append(Intert)

        del start_index, end_index, inter_arr, intra_arr, inter_arr1, intra_arr1, inter_arr2, intra_arr2, new_df

    return avg_Intra, avg_Inter, tavg_Intra, tavg_Inter

In [ ]:
for i in range(len(packageArr)):
    #패키지 별 outlier 제거
    high = qdf.loc[qdf['packageName'] == packageArr[i]]['P_highest'].reset_index(drop=True)
    if high.empty:
        del high
        continue;

    low = qdf.loc[qdf['packageName'] == packageArr[i]]['P_lowest'].reset_index(drop=True)
    highest = high[0]
    originlowest = low[0]
    if originlowest <= 0:
        lowest = 1
    else:
        lowest = originlowest

    #pacakge 이름에 해당하는 행들과 Device Event를 가져옴
    bufferdf = df[(df['packageName']==packageArr[i]) | (df['datumType'] == 'DEVICE_EVENT')]
    if bufferdf['timeTaken'].isnull().all():
        del bufferdf, high, low, highest, lowest, originlowest
        continue;

    #각 view의 시작점과 종결점을 잡기위한 loopdf를 만드는 작업
    classdf = bufferdf.loc[(bufferdf['deviceType'].isin(['typeWindowStateChanged', 'typeViewFocused'])),['timestamp', 'packageName', 'deviceType', 'className', 'type']]
    eventdf = bufferdf.loc[(bufferdf['type'].isin(['SCREEN_OFF', 'SCREEN_ON', 'USER_PRESENT'])),['timestamp', 'packageName', 'deviceType', 'className', 'type']]
    loopdf = classdf.append(eventdf)
    if loopdf.empty:
        del bufferdf, eventdf, classdf, loopdf, high, low, highest, lowest, originlowest
        continue;

    loopdf = loopdf.sort_index()
    ind_list = loopdf.index
    classArr = loopdf['className'].unique() #loopdf에서 모든 class 이름을 가져와 list로 만듦

    #package별 typing metric을 만들기 위한 전처리
    pavgCPS = []
    pminCPS = []
    pmidCPS = []
    pmaxCPS = []
    pavgCPM = []
    pminCPM = []
    pmidCPM = []
    pmaxCPM = []
    pavgWPS = []
    pminWPS = []
    pmidWPS = []
    pmaxWPS = []
    pavgWPM = []
    pminWPM = []
    pmidWPM = []
    pmaxWPM = []
    pavgKSPS = []
    pminKSPS = []
    pmidKSPS = []
    pmaxKSPS = []
    pavgKSPC = []
    pminKSPC = []
    pmidKSPC = []
    pmaxKSPC = []
    pavg_Intra = []
    pavg_Inter = []
    tavg_Intra = []
    tavg_Inter = []

    #class 별 typing metric을 추출한 bufdf와 pacakge 별 typing metric을 저장한 list를 받아옴
    bufdf, pavgCPS, pavgCPM, pavgWPS, pavgWPM, pavgKSPS, pavgKSPC, pminCPS, pmidCPS, pmaxCPS, pminCPM , pmidCPM, pmaxCPM, pminWPS,
    pmidWPS, pmaxWPS, pminWPM, pmidWPM , pmaxWPM, pminKSPS, pmidKSPS, pmaxKSPS, pminKSPC, pmidKSPC, pmaxKSPC = get_caver(bufferdf, loopdf, classArr)

    #package 별, 전체인원 별 아웃라이어를 제거한 Intracharacter, Intercharacter time 가져옴
    pavg_Intra, pavg_Inter, tavg_Intra, tavg_Inter = get_paver(ind_list, bufferdf, highest, lowest)

    #sumdf에 bufdf 추가
    sumdf = sumdf.append(bufdf, ignore_index=True)
    sumdf['packageName'] = sumdf['packageName'].fillna(packageArr[i])

    #한 tester 대한 전체 typing metric을 구하기 위한 전처리
    tavgCPS = pavgCPS + tavgCPS
    tavgCPM = pavgCPM + tavgCPM
    tavgWPS = pavgWPS + tavgWPS
    tavgWPM = pavgWPM + tavgWPM
    tavgKSPS = pavgKSPS + tavgKSPS
    tavgKSPC = pavgKSPC + tavgKSPC
    tminCPS = pminCPS + tminCPS
    tmidCPS = pmidCPS + tmidCPS
    tmaxCPS = pmaxCPS + tmaxCPS
    tminCPM = pminCPM + tminCPM
    tmidCPM = pmidCPM + tmidCPM
    tmaxCPM = pmaxCPM + tmaxCPM
    tminWPS = pminWPS + tminWPS
    tmidWPS = pmidWPS + tmidWPS
    tmaxWPS = pmaxWPS + tmaxWPS
    tminWPM = pminWPM + tminWPM
    tmidWPM = pmidWPM + tmidWPM
    tmaxWPM = pmaxWPM + tmaxWPM
    tminKSPS = pminKSPS + tminKSPS
    tmidKSPS = pmidKSPS + tmidKSPS
    tmaxKSPS = pmaxKSPS + tmaxKSPS
    tminKSPC = pminKSPC + tminKSPC
    tmidKSPC = pmidKSPC + tmidKSPC
    tmaxKSPC = pmaxKSPC + tmaxKSPC

    pavg_Intra = [x for x in pavg_Intra if x is not None]
    pavg_Inter = [x for x in pavg_Inter if x is not None]
    pavg_Intra = [x for x in pavg_Intra if x > 0]
    pavg_Inter = [x for x in pavg_Inter if x > 0]
    tavg_Intra = [x for x in pavg_Intra if x is not None]
    tavg_Inter = [x for x in pavg_Inter if x is not None]

    avgInter = avgInter + tavg_Inter
    avgIntra = avgIntra + tavg_Intra

    if len(pavg_Inter) <= 0:
            del bufferdf, classdf, eventdf, loopdf, pavgCPS, pavgCPM, pavgWPS, pavgWPM, pavgKSPS, pavgKSPC, pavg_Intra, pavg_Inter, lowest, highest, low, high, pminCPS, pmidCPS, pmaxCPS, pminCPM , pmidCPM, pmaxCPM, pminWPS, pmidWPS, pmaxWPS, pminWPM, pmidWPM , pmaxWPM, pminKSPS, pmidKSPS, pmaxKSPS, pminKSPC, pmidKSPC, pmaxKSPC
            continue;

    #한 tester의 각 package 별 typing metric의 통계치 추출해서 packagedf에 저장
    newRow = {'tester': tester, 'packageName': packageArr[i], 'PavgCPS': np.mean(pavgCPS), 'PminCPS': np.mean(pminCPS),
              'PmidCPS': np.mean(pmidCPS), 'PmaxCPS': np.mean(pmaxCPS), 'PavgCPM': np.mean(pavgCPM), 'PminCPM': np.mean(pminCPM),
              'PmidCPM': np.mean(pmidCPM), 'PmaxCPM': np.mean(pmaxCPM), 'PavgWPS': np.mean(pavgWPS), 'PminWPS': np.mean(pminWPS),
              'PmidWPS': np.mean(pmidWPS), 'PmaxWPS': np.mean(pmaxWPS), 'PavgWPM': np.mean(pavgWPM), 'PminWPM': np.mean(pminWPM),
              'PmidWPM': np.mean(pmidWPM), 'PmaxWPM': np.mean(pmaxWPM), 'PavgKSPS': np.mean(pavgKSPS), 'PminKSPS': np.mean(pminKSPS),
              'PmidKSPS': np.mean(pmidKSPS), 'PmaxKSPS': np.mean(pmaxKSPS), 'PavgKSPC': np.mean(pavgKSPC),
              'PminKSPC': np.mean(pminKSPC), 'PmidKSPC': np.mean(pmidKSPC), 'PmaxKSPC': np.mean(pmaxKSPC),
              'PavgIntra': np.mean(pavg_Intra), 'PminIntra': np.min(pavg_Intra), 'PmidIntra': np.median(pavg_Intra),
              'PmaxIntra': np.max(pavg_Intra), 'PavgInter': np.mean(pavg_Inter), 'PminInter': np.min(pavg_Inter),
              'PmidInter': np.median(pavg_Inter), 'PmaxInter': np.max(pavg_Inter)}
    packagedf = packagedf.append(newRow, ignore_index=True)


    del bufferdf, classdf, eventdf, loopdf, newRow, pavgCPS, pavgCPM, pavgWPS, pavgWPM, pavgKSPS, pavgKSPC, pavg_Intra, pavg_Inter, lowest, highest, low, high, pminCPS, pmidCPS, pmaxCPS, pminCPM , pmidCPM, pmaxCPM, pminWPS, pmidWPS, pmaxWPS, pminWPM, pmidWPM , pmaxWPM, pminKSPS, pmidKSPS, pmaxKSPS, pminKSPC, pmidKSPC, pmaxKSPC

c:\users\dongh\python\lib\site-packages\ipykernel_launcher.py:132: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\dongh\python\lib\site-packages\ipykernel_launcher.py:136: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\dongh\python\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\dongh\python\lib\site-packages\ipykernel_launcher.py:129: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\dongh\python\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: invalid value encountered in double_scalars
c:\users\dongh\python\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\dongh\python\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### 결과

In [ ]:
sumdf

,packageName,className,CavgCPS,CminCPS,CmidCPS,CmaxCPS,CavgCPM,CminCPM,CmidCPM,CmaxCPM,CavgWPS,CminWPS,CmidWPS,CmaxWPS,CavgWPM,CminWPM,CmidWPM,CmaxWPM,CavgKSPS,CminKSPS,CmidKSPS,CmaxKSPS,CavgKSPC,CminKSPC,CmidKSPC,CmaxKSPC,CavgIntra,CminIntra,CmidIntra,CmaxIntra,CavgInter,CminInter,CmidInter,CmaxInter
0,com.sec.android.app.launcher,android.widget.FrameLayout,1.323232,1.313131,1.323232,1.333333,79.393939,78.787879,79.393939,80.000000,0.264646,0.262626,0.264646,0.266667,15.878788,15.757576,15.878788,16.000000,1.959596,1.919192,1.959596,2.000000,1.345238,1.333333,1.345238,1.357143,89.245614,87.157895,89.245614,91.333333,88.245614,86.157895,88.245614,90.333333
1,com.sec.android.app.launcher,android.widget.EditText,2.068966,2.068966,2.068966,2.068966,124.137931,124.137931,124.137931,124.137931,0.413793,0.413793,0.413793,0.413793,24.827586,24.827586,24.827586,24.827586,1.264368,0.000000,0.000000,3.793103,1.571429,1.571429,1.571429,1.571429,77.909091,77.909091,77.909091,77.909091,76.909091,76.909091,76.909091,76.909091
2,com.kakao.talk,com.kakao.talk.activity.chatroom.ChatRoomHolde...,5.395147,1.826484,5.000000,10.000000,323.708849,109.589041,300.000000,600.000000,1.079029,0.365297,1.000000,2.000000,64.741770,21.917808,60.000000,120.000000,6.777002,2.146119,6.195330,15.000000,1.140711,1.000000,1.083477,1.500000,100.405469,63.666667,102.766993,157.851064,99.405469,62.666667,101.766993,156.851064
3,com.kakao.talk,android.widget.MultiAutoCompleteTextView,4.541916,0.000000,4.322917,12.000000,272.514935,0.000000,259.375000,720.000000,0.908383,0.000000,0.864583,2.400000,54.502987,0.000000,51.875000,144.000000,5.220637,0.000000,5.197164,14.000000,1.156676,1.000000,1.111111,2.000000,106.619346,16.000000,104.545455,219.863636,105.619346,15.000000,103.545455,218.863636
4,com.kakao.talk,android.webkit.WebView,4.082126,2.608696,4.082126,5.555556,244.927536,156.521739,244.927536,333.333333,0.816425,0.521739,0.816425,1.111111,48.985507,31.304348,48.985507,66.666667,3.236715,0.000000,3.043478,6.666667,1.000000,1.000000,1.000000,1.000000,92.428571,6.000000,117.285714,154.000000,131.928571,110.857143,131.928571,153.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,com.google.android.keep,android.widget.EditText,2.798424,0.491803,0.980392,6.923077,167.905447,29.508197,58.823529,415.384615,0.559685,0.098361,0.196078,1.384615,33.581089,5.901639,11.764706,83.076923,4.559115,1.176471,4.683841,7.692308,3.200658,1.000000,2.026316,7.750000,42.539606,26.935484,42.333333,58.350000,41.539606,25.935484,41.333333,57.350000
73,com.samsung.android.app.reminder,android.widget.AutoCompleteTextView,5.857843,3.382353,5.857843,8.333333,351.470588,202.941176,351.470588,500.000000,1.171569,0.676471,1.171569,1.666667,70.294118,40.588235,70.294118,100.000000,6.578431,3.823529,6.578431,9.333333,1.080128,1.076923,1.080128,1.083333,14.447802,3.538462,14.447802,25.357143,13.447802,2.538462,13.447802,24.357143
74,com.reddit.frontpage,android.widget.EditText,2.500000,2.500000,2.500000,2.500000,150.000000,150.000000,150.000000,150.000000,0.500000,0.500000,0.500000,0.500000,30.000000,30.000000,30.000000,30.000000,5.000000,5.000000,5.000000,5.000000,1.333333,1.333333,1.333333,1.333333,244.000000,244.000000,244.000000,244.000000,189.750000,189.750000,189.750000,189.750000
75,com.kiwoom.heromts,android.widget.EditText,6.458333,1.250000,6.458333,11.666667,387.500000,75.000000,387.500000,700.000000,1.291667,0.250000,1.291667,2.333333,77.500000,15.000000,77.500000,140.000000,8.281250,1.562500,8.281250,15.000000,1.062500,1.000000,1.062500,1.125000,67.555556,67.555556,67.555556,67.555556,66.555556,66.555556,66.555556,66.555556


In [ ]:
packagedf

,tester,packageName,PavgCPS,PminCPS,PmidCPS,PmaxCPS,PavgCPM,PminCPM,PmidCPM,PmaxCPM,PavgWPS,PminWPS,PmidWPS,PmaxWPS,PavgWPM,PminWPM,PmidWPM,PmaxWPM,PavgKSPS,PminKSPS,PmidKSPS,PmaxKSPS,PavgKSPC,PminKSPC,PmidKSPC,PmaxKSPC,PavgIntra,PminIntra,PmidIntra,PmaxIntra,PavgInter,PminInter,PmidInter,PmaxInter
0,tester6,com.sec.android.app.launcher,1.571810,1.691048,1.696099,1.701149,94.308603,101.462905,101.765935,102.068966,0.314362,0.338210,0.339220,0.340230,18.861721,20.292581,20.353187,20.413793,1.542459,0.959596,0.979798,2.896552,1.420635,1.452381,1.458333,1.464286,99.436470,87.157895,91.333333,119.818182,98.436470,86.157895,90.333333,118.818182
1,tester6,com.kakao.talk,4.965566,2.217696,4.161578,9.179614,297.933989,133.061761,249.694697,550.776836,0.993113,0.443539,0.832316,1.835923,59.586798,26.612352,49.938939,110.155367,5.767112,1.912172,5.080746,11.431548,1.162251,1.054754,1.111320,1.528961,93.683704,6.000000,94.105263,178.454545,93.075395,15.000000,93.118897,177.454545
2,tester6,com.android.systemui,4.081664,1.898734,4.230769,6.111111,244.899850,113.924051,253.846154,366.666667,0.816333,0.379747,0.846154,1.222222,48.979970,22.784810,50.769231,73.333333,4.609321,2.151899,4.615385,6.666667,1.069690,1.000000,1.000000,1.400000,144.653708,117.911765,144.583333,171.833333,143.653708,116.911765,143.583333,170.833333
3,tester6,com.nhn.android.search,4.518161,3.928571,5.130952,6.833333,271.089667,235.714286,307.857143,410.000000,0.903632,0.785714,1.026190,1.366667,54.217933,47.142857,61.571429,82.000000,5.978406,4.568151,6.448413,8.666667,2.542659,1.071429,1.071429,7.000000,155.610168,83.500000,163.743421,234.111111,139.849934,82.500000,150.164474,189.545455
4,tester6,com.sec.android.app.clockpackage,6.600000,6.600000,6.600000,6.600000,396.000000,396.000000,396.000000,396.000000,1.320000,1.320000,1.320000,1.320000,79.200000,79.200000,79.200000,79.200000,7.600000,7.600000,7.600000,7.600000,1.117647,1.117647,1.117647,1.117647,113.210526,113.210526,113.210526,113.210526,112.210526,112.210526,112.210526,112.210526
5,tester6,com.android.settings,1.840979,1.344538,1.553398,2.625000,110.458717,80.672269,93.203883,157.500000,0.368196,0.268908,0.310680,0.525000,22.091743,16.134454,18.640777,31.500000,6.204521,2.912621,3.809874,14.285714,6.501337,1.181818,2.411765,20.000000,190.436667,160.666667,170.750000,272.000000,170.124359,115.850000,159.666667,245.269231
6,tester6,com.google.android.youtube,5.104897,0.833333,4.350000,11.250000,306.293823,50.000000,261.000000,675.000000,1.020979,0.166667,0.870000,2.250000,61.258765,10.000000,52.200000,135.000000,6.764800,1.860465,6.250000,15.000000,1.227398,1.050000,1.125000,2.000000,143.141495,70.416667,123.461538,287.833333,141.658736,69.416667,122.461538,286.833333
7,tester6,com.kakao.taxi,9.259259,6.666667,8.888889,12.222222,555.555556,400.000000,533.333333,733.333333,1.851852,1.333333,1.777778,2.444444,111.111111,80.000000,106.666667,146.666667,11.407407,8.666667,11.111111,14.444444,1.125421,1.083333,1.111111,1.181818,92.353846,72.076923,92.600000,112.384615,91.353846,71.076923,91.600000,111.384615
8,tester6,com.twitter.android,2.574338,1.730769,2.746897,4.562500,154.460306,103.846154,164.813801,273.750000,0.514868,0.346154,0.549379,0.912500,30.892061,20.769231,32.962760,54.750000,4.360054,2.726648,3.674242,7.812500,1.305285,1.055556,1.112903,1.766667,153.498793,88.682927,163.368421,234.142857,144.647917,87.682927,146.797619,233.142857
9,tester6,com.android.chrome,4.094487,3.597257,5.934849,8.213141,245.669227,215.835440,356.090924,492.788462,0.818897,0.719451,1.186970,1.642628,49.133845,43.167088,71.218185,98.557692,5.600298,5.528347,8.460071,12.916667,1.248469,1.062500,1.186928,1.937500,144.467266,0.500000,143.694444,456.000000,130.100883,1.000000,129.762222,307.600000


tester의 전체 view에 대한 typing metric 추출 및 teseterdf에 추가

In [ ]:
testerdf.loc[0] = [tester, np.mean(tavgCPS), np.mean(tminCPS), np.mean(tmidCPS), np.mean(tmaxCPS), np.mean(tavgCPM), np.mean(tminCPM),
                   np.mean(tmidCPM), np.mean(tmaxCPM), np.mean(tavgWPS), np.mean(tminWPS), np.mean(tmidWPS), np.mean(tmaxWPS),
                   np.mean(tavgWPM), np.mean(tminWPM), np.mean(tmidWPM), np.mean(tmaxWPM), np.mean(tavgKSPS), np.mean(tminKSPS),
                   np.mean(tmidKSPS), np.mean(tmaxKSPS), np.mean(tavgKSPC), np.mean(tminKSPC), np.mean(tmidKSPC), np.mean(tmaxKSPC),
                   np.mean(avgIntra), np.min(avgIntra), np.median(avgIntra), np.max(avgIntra), np.mean(avgInter), np.min(avgInter),
                   np.median(avgInter), np.max(avgInter)]

In [ ]:
testerdf

,tester,avgCPS,minCPS,midCPS,maxCPS,avgCPM,minCPM,midCPM,maxCPM,avgWPS,minWPS,midWPS,maxWPS,avgWPM,minWPM,midWPM,maxWPM,avgKSPS,minKSPS,midKSPS,maxKSPS,avgKSPC,minKSPC,midKSPC,maxKSPC,avgIntra,minIntra,midIntra,maxIntra,avgInter,minInter,midInter,maxInter
0,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0


tester기준으로 packagedf와 합침

In [ ]:
df1 = pd.merge(left=testerdf, right=packagedf, how="left", on=["tester"])
df1

,tester,avgCPS,minCPS,midCPS,maxCPS,avgCPM,minCPM,midCPM,maxCPM,avgWPS,minWPS,midWPS,maxWPS,avgWPM,minWPM,midWPM,maxWPM,avgKSPS,minKSPS,midKSPS,maxKSPS,avgKSPC,minKSPC,midKSPC,maxKSPC,avgIntra,minIntra,midIntra,maxIntra,avgInter,minInter,midInter,maxInter,packageName,PavgCPS,PminCPS,PmidCPS,PmaxCPS,PavgCPM,PminCPM,PmidCPM,PmaxCPM,PavgWPS,PminWPS,PmidWPS,PmaxWPS,PavgWPM,PminWPM,PmidWPM,PmaxWPM,PavgKSPS,PminKSPS,PmidKSPS,PmaxKSPS,PavgKSPC,PminKSPC,PmidKSPC,PmaxKSPC,PavgIntra,PminIntra,PmidIntra,PmaxIntra,PavgInter,PminInter,PmidInter,PmaxInter
0,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.sec.android.app.launcher,1.571810,1.691048,1.696099,1.701149,94.308603,101.462905,101.765935,102.068966,0.314362,0.338210,0.339220,0.340230,18.861721,20.292581,20.353187,20.413793,1.542459,0.959596,0.979798,2.896552,1.420635,1.452381,1.458333,1.464286,99.436470,87.157895,91.333333,119.818182,98.436470,86.157895,90.333333,118.818182
1,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.kakao.talk,4.965566,2.217696,4.161578,9.179614,297.933989,133.061761,249.694697,550.776836,0.993113,0.443539,0.832316,1.835923,59.586798,26.612352,49.938939,110.155367,5.767112,1.912172,5.080746,11.431548,1.162251,1.054754,1.111320,1.528961,93.683704,6.000000,94.105263,178.454545,93.075395,15.000000,93.118897,177.454545
2,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.android.systemui,4.081664,1.898734,4.230769,6.111111,244.899850,113.924051,253.846154,366.666667,0.816333,0.379747,0.846154,1.222222,48.979970,22.784810,50.769231,73.333333,4.609321,2.151899,4.615385,6.666667,1.069690,1.000000,1.000000,1.400000,144.653708,117.911765,144.583333,171.833333,143.653708,116.911765,143.583333,170.833333
3,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.nhn.android.search,4.518161,3.928571,5.130952,6.833333,271.089667,235.714286,307.857143,410.000000,0.903632,0.785714,1.026190,1.366667,54.217933,47.142857,61.571429,82.000000,5.978406,4.568151,6.448413,8.666667,2.542659,1.071429,1.071429,7.000000,155.610168,83.500000,163.743421,234.111111,139.849934,82.500000,150.164474,189.545455
4,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.sec.android.app.clockpackage,6.600000,6.600000,6.600000,6.600000,396.000000,396.000000,396.000000,396.000000,1.320000,1.320000,1.320000,1.320000,79.200000,79.200000,79.200000,79.200000,7.600000,7.600000,7.600000,7.600000,1.117647,1.117647,1.117647,1.117647,113.210526,113.210526,113.210526,113.210526,112.210526,112.210526,112.210526,112.210526
5,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.android.setting

package 기준으로 sumdf 합침

In [ ]:
df2 = pd.merge(left=df1, right=sumdf, how="left", on=["packageName"])
df2

,tester,avgCPS,minCPS,midCPS,maxCPS,avgCPM,minCPM,midCPM,maxCPM,avgWPS,minWPS,midWPS,maxWPS,avgWPM,minWPM,midWPM,maxWPM,avgKSPS,minKSPS,midKSPS,maxKSPS,avgKSPC,minKSPC,midKSPC,maxKSPC,avgIntra,minIntra,midIntra,maxIntra,avgInter,minInter,midInter,maxInter,packageName,PavgCPS,PminCPS,PmidCPS,PmaxCPS,PavgCPM,PminCPM,PmidCPM,PmaxCPM,PavgWPS,PminWPS,PmidWPS,PmaxWPS,PavgWPM,PminWPM,PmidWPM,PmaxWPM,PavgKSPS,PminKSPS,PmidKSPS,PmaxKSPS,PavgKSPC,PminKSPC,PmidKSPC,PmaxKSPC,PavgIntra,PminIntra,PmidIntra,PmaxIntra,PavgInter,PminInter,PmidInter,PmaxInter,className,CavgCPS,CminCPS,CmidCPS,CmaxCPS,CavgCPM,CminCPM,CmidCPM,CmaxCPM,CavgWPS,CminWPS,CmidWPS,CmaxWPS,CavgWPM,CminWPM,CmidWPM,CmaxWPM,CavgKSPS,CminKSPS,CmidKSPS,CmaxKSPS,CavgKSPC,CminKSPC,CmidKSPC,CmaxKSPC,CavgIntra,CminIntra,CmidIntra,CmaxIntra,CavgInter,CminInter,CmidInter,CmaxInter
0,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.sec.android.app.launcher,1.571810,1.691048,1.696099,1.701149,94.308603,101.462905,101.765935,102.068966,0.314362,0.338210,0.339220,0.340230,18.861721,20.292581,20.353187,20.413793,1.542459,0.959596,0.979798,2.896552,1.420635,1.452381,1.458333,1.464286,99.436470,87.157895,91.333333,119.818182,98.436470,86.157895,90.333333,118.818182,android.widget.FrameLayout,1.323232,1.313131,1.323232,1.333333,79.393939,78.787879,79.393939,80.000000,0.264646,0.262626,0.264646,0.266667,15.878788,15.757576,15.878788,16.000000,1.959596,1.919192,1.959596,2.000000,1.345238,1.333333,1.345238,1.357143,89.245614,87.157895,89.245614,91.333333,88.245614,86.157895,88.245614,90.333333
1,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.sec.android.app.launcher,1.571810,1.691048,1.696099,1.701149,94.308603,101.462905,101.765935,102.068966,0.314362,0.338210,0.339220,0.340230,18.861721,20.292581,20.353187,20.413793,1.542459,0.959596,0.979798,2.896552,1.420635,1.452381,1.458333,1.464286,99.436470,87.157895,91.333333,119.818182,98.436470,86.157895,90.333333,118.818182,android.widget.EditText,2.068966,2.068966,2.068966,2.068966,124.137931,124.137931,124.137931,124.137931,0.413793,0.413793,0.413793,0.413793,24.827586,24.827586,24.827586,24.827586,1.264368,0.000000,0.000000,3.793103,1.571429,1.571429,1.571429,1.571429,77.909091,77.909091,77.909091,77.909091,76.909091,76.909091,76.909091,76.909091
2,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1018.0,120.963521,1.0,105.0,700.0,com.kakao.talk,4.965566,2.217696,4.161578,9.179614,297.933989,133.061761,249.694697,550.776836,0.993113,0.443539,0.832316,1.835923,59.586798,26.612352,49.938939,110.155367,5.767112,1.912172,5.080746,11.431548,1.162251,1.054754,1.111320,1.528961,93.683704,6.000000,94.105263,178.454545,93.075395,15.000000,93.118897,177.454545,com.kakao.talk.activity.chatroom.ChatRoomHolde...,5.395147,1.826484,5.000000,10.000000,323.708849,109.589041,300.000000,600.000000,1.079029,0.365297,1.000000,2.000000,64.741770,21.917808,60.000000,120.000000,6.777002,2.146119,6.195330,15.000000,1.140711,1.000000,1.083477,1.500000,100.405469,63.666667,102.766993,157.851064,99.405469,62.666667,101.766993,156.851064
3,tester6,4.592577,3.255349,4.643953,6.49049,275.554593,195.320925,278.637183,389.429403,0.918515,0.65107,0.928791,1.298098,55.110919,39.064185,55.727437,77.885881,5.921246,4.120904,6.399919,9.407375,1.362001,1.16596,1.270488,2.550548,128.639432,0.5,108.25,1

In [ ]:
df2 = df2.round(2) #Dataframe 소수점 2자리까지만 보이게

In [ ]:
df2

,tester,avgCPS,minCPS,midCPS,maxCPS,avgCPM,minCPM,midCPM,maxCPM,avgWPS,minWPS,midWPS,maxWPS,avgWPM,minWPM,midWPM,maxWPM,avgKSPS,minKSPS,midKSPS,maxKSPS,avgKSPC,minKSPC,midKSPC,maxKSPC,avgIntra,minIntra,midIntra,maxIntra,avgInter,minInter,midInter,maxInter,packageName,PavgCPS,PminCPS,PmidCPS,PmaxCPS,PavgCPM,PminCPM,PmidCPM,PmaxCPM,PavgWPS,PminWPS,PmidWPS,PmaxWPS,PavgWPM,PminWPM,PmidWPM,PmaxWPM,PavgKSPS,PminKSPS,PmidKSPS,PmaxKSPS,PavgKSPC,PminKSPC,PmidKSPC,PmaxKSPC,PavgIntra,PminIntra,PmidIntra,PmaxIntra,PavgInter,PminInter,PmidInter,PmaxInter,className,CavgCPS,CminCPS,CmidCPS,CmaxCPS,CavgCPM,CminCPM,CmidCPM,CmaxCPM,CavgWPS,CminWPS,CmidWPS,CmaxWPS,CavgWPM,CminWPM,CmidWPM,CmaxWPM,CavgKSPS,CminKSPS,CmidKSPS,CmaxKSPS,CavgKSPC,CminKSPC,CmidKSPC,CmaxKSPC,CavgIntra,CminIntra,CmidIntra,CmaxIntra,CavgInter,CminInter,CmidInter,CmaxInter
0,tester6,4.59,3.26,4.64,6.49,275.55,195.32,278.64,389.43,0.92,0.65,0.93,1.3,55.11,39.06,55.73,77.89,5.92,4.12,6.4,9.41,1.36,1.17,1.27,2.55,128.64,0.5,108.25,1018.0,120.96,1.0,105.0,700.0,com.sec.android.app.launcher,1.57,1.69,1.70,1.70,94.31,101.46,101.77,102.07,0.31,0.34,0.34,0.34,18.86,20.29,20.35,20.41,1.54,0.96,0.98,2.90,1.42,1.45,1.46,1.46,99.44,87.16,91.33,119.82,98.44,86.16,90.33,118.82,android.widget.FrameLayout,1.32,1.31,1.32,1.33,79.39,78.79,79.39,80.00,0.26,0.26,0.26,0.27,15.88,15.76,15.88,16.00,1.96,1.92,1.96,2.00,1.35,1.33,1.35,1.36,89.25,87.16,89.25,91.33,88.25,86.16,88.25,90.33
1,tester6,4.59,3.26,4.64,6.49,275.55,195.32,278.64,389.43,0.92,0.65,0.93,1.3,55.11,39.06,55.73,77.89,5.92,4.12,6.4,9.41,1.36,1.17,1.27,2.55,128.64,0.5,108.25,1018.0,120.96,1.0,105.0,700.0,com.sec.android.app.launcher,1.57,1.69,1.70,1.70,94.31,101.46,101.77,102.07,0.31,0.34,0.34,0.34,18.86,20.29,20.35,20.41,1.54,0.96,0.98,2.90,1.42,1.45,1.46,1.46,99.44,87.16,91.33,119.82,98.44,86.16,90.33,118.82,android.widget.EditText,2.07,2.07,2.07,2.07,124.14,124.14,124.14,124.14,0.41,0.41,0.41,0.41,24.83,24.83,24.83,24.83,1.26,0.00,0.00,3.79,1.57,1.57,1.57,1.57,77.91,77.91,77.91,77.91,76.91,76.91,76.91,76.91
2,tester6,4.59,3.26,4.64,6.49,275.55,195.32,278.64,389.43,0.92,0.65,0.93,1.3,55.11,39.06,55.73,77.89,5.92,4.12,6.4,9.41,1.36,1.17,1.27,2.55,128.64,0.5,108.25,1018.0,120.96,1.0,105.0,700.0,com.kakao.talk,4.97,2.22,4.16,9.18,297.93,133.06,249.69,550.78,0.99,0.44,0.83,1.84,59.59,26.61,49.94,110.16,5.77,1.91,5.08,11.43,1.16,1.05,1.11,1.53,93.68,6.00,94.11,178.45,93.08,15.00,93.12,177.45,com.kakao.talk.activity.chatroom.ChatRoomHolde...,5.40,1.83,5.00,10.00,323.71,109.59,300.00,600.00,1.08,0.37,1.00,2.00,64.74,21.92,60.00,120.00,6.78,2.15,6.20,15.00,1.14,1.00,1.08,1.50,100.41,63.67,102.77,157.85,99.41,62.67,101.77,156.85
3,tester6,4.59,3.26,4.64,6.49,275.55,195.32,278.64,389.43,0.92,0.65,0.93,1.3,55.11,39.06,55.73,77.89,5.92,4.12,6.4,9.41,1.36,1.17,1.27,2.55,128.64,0.5,108.25,1018.0,120.96,1.0,105.0,700.0,com.kakao.talk,4.97,2.22,4.16,9.18,297.93,133.06,249.69,550.78,0.99,0.44,0.83,1.84,59.59,26.61,49.94,110.16,5.77,1.91,5.08,11.43,1.16,1.05,1.11,1.53,93.68,6.00,94.11,178.45,93.08,15.00,93.12,177.45,android.widget.MultiAutoCompleteTextView,4.54,0.00,4.32,12.00,272.51,0.00,259.38,720.00,0.91,0.00,0.86,2.40,54.50,0.00,51.88,144.00,5.22,0.00,5.20,14.00,1.16,1.00,1.11,2.00,106.62,16.00,104.55,219.86,105.62,15.00,103.55,218.86
4,tester6,4.59,3.26,4.64,6.49,275.55,195.32,278.64,389.43,0.92,0.65,0.93,1.3,55.11,39.06,55.73,77.89,5.92,4.12,6.4,9.41,1.36,1.17,1.27,2.55,128.64,0.5,108.25,1018.0,120.96,1.0,105.0,700.0,com.kakao.talk,4.97,2.22,4.16,9.18,297.93,133.06,249.69,550.78,0.99,0.44,0.83,1.84,59.59,26.61,49.94,110.16,5.77,1.91,5.08,11.43,1.16,1.05,1.11,1.53,93.68,6.00,94.11,178.45,93.08,15.00,93.12,177.45,android.webkit.WebView,4.08,2.61,4.08,5.56,244.93,156.52,244.93,333.33,0.82,0.52,0.82,1.11,48.99,31.30,48.99,66.67,3.24,0.00,3.04,6.67,1.00,1.00,1.00,1.00,92.43,6.00,117.29,154.00,131.93,110.86,131.93,153.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### csv 파일로 저장

In [ ]:
df2.to_csv("C:/Users/dongh/abc/thesis/tester6_metric.csv", index=None, encoding='utf-8-sig')